## Create the dataframe

In [56]:
from pyspark.sql import SparkSession

# Because this is a local script, we need to create a Spark session.
# In a Databricks notebook, the Spark session is created automatically. We only need to state %%pyspark at the top of the cell.

spark = SparkSession.builder \
    .appName("ReadCSVExample") \
    .getOrCreate()

df = spark.read.format("csv").option("header", "false").load("orders/2019.csv")
df.show() # df.show() is a standard PySpark method, while display() is a method from Databricks notebooks.

ConnectionRefusedError: [Errno 61] Connection refused

In [24]:
from pyspark.sql.types import *

orderSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("Email", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("Tax", FloatType())
])

df = spark.read.format("csv").schema(orderSchema).load("orders/*.csv")

df.show(5)


25/07/30 13:32:54 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: orders/*.csv.
java.io.FileNotFoundException: File orders/*.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:917)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1238)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:907)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:56)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:381)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource.org$apache$spark$sql$catalyst$analysis$ResolveDataSource$$loadV1BatchSource(ResolveDataSource.scala:143)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource$$anonfun$

+----------------+--------------------+----------+--------------+--------------------+--------------------+--------+---------+--------+
|SalesOrderNumber|SalesOrderLineNumber| OrderDate|  CustomerName|               Email|                Item|Quantity|UnitPrice|     Tax|
+----------------+--------------------+----------+--------------+--------------------+--------------------+--------+---------+--------+
|         SO49171|                   1|2021-01-01| Mariah Foster|mariah21@adventur...|  Road-250 Black, 48|       1|2181.5625| 174.525|
|         SO49172|                   1|2021-01-01|  Brian Howard|brian23@adventure...|    Road-250 Red, 44|       1|  2443.35| 195.468|
|         SO49173|                   1|2021-01-01| Linda Alvarez|linda19@adventure...|Mountain-200 Silv...|       1|2071.4197|165.7136|
|         SO49174|                   1|2021-01-01|Gina Hernandez|gina4@adventure-w...|Mountain-200 Silv...|       1|2071.4197|165.7136|
|         SO49178|                   1|2021-01-0

## Filtering and Aggregating

In [25]:
customers = df['CustomerName', 'Email']

print(customers.count())
print(customers.distinct().count())

customers.show(5)

32718
12427
+--------------+--------------------+
|  CustomerName|               Email|
+--------------+--------------------+
| Mariah Foster|mariah21@adventur...|
|  Brian Howard|brian23@adventure...|
| Linda Alvarez|linda19@adventure...|
|Gina Hernandez|gina4@adventure-w...|
|     Beth Ruiz|beth4@adventure-w...|
+--------------+--------------------+
only showing top 5 rows


In [26]:
customers = df.select("CustomerName", "Email").where(df['Item']=='Road-250 Red, 52')
print(customers.count())
print(customers.distinct().count())

customers.show(5)

133
133
+----------------+--------------------+
|    CustomerName|               Email|
+----------------+--------------------+
|    Margaret Guo|margaret24@advent...|
|         Cara Xu|cara8@adventure-w...|
|  Alejandro Raji|alejandro46@adven...|
|      Jaime Diaz|jaime3@adventure-...|
|Bridget Andersen|bridget15@adventu...|
+----------------+--------------------+
only showing top 5 rows


In [27]:
productSales = df.select("Item", "Quantity").groupBy("Item").sum()

productSales.show(5)

+--------------------+-------------+
|                Item|sum(Quantity)|
+--------------------+-------------+
|Mountain-200 Blac...|          388|
|Touring-1000 Yell...|           74|
|Touring-1000 Blue...|           67|
|Short-Sleeve Clas...|          216|
|Women's Mountain ...|          146|
+--------------------+-------------+
only showing top 5 rows


In [28]:
productSales = df.select("Item", "Quantity").groupBy("Item").sum()
productSales.show(5)

+--------------------+-------------+
|                Item|sum(Quantity)|
+--------------------+-------------+
|Mountain-200 Blac...|          388|
|Touring-1000 Yell...|           74|
|Touring-1000 Blue...|           67|
|Short-Sleeve Clas...|          216|
|Women's Mountain ...|          146|
+--------------------+-------------+
only showing top 5 rows


In [29]:
from pyspark.sql.functions import *

# This creates a new DataFrame with a single column called 'Year', which contains the year extracted from the 'OrderDate' column.

yearlySales = df.select(year(col("OrderDate")).alias("Year")).groupBy("Year").count().orderBy("Year")

yearlySales.show()

+----+-----+
|Year|count|
+----+-----+
|2019| 1201|
|2020| 2733|
|2021|28784|
+----+-----+



## Transform and Save

In [30]:
# Create Year and Month columns
transformed_df = df.withColumn("Year", year(col("OrderDate"))).withColumn("Month", month(col("OrderDate")))

# Create the new FirstName and LastName fields
transformed_df = transformed_df.withColumn("FirstName", split(col("CustomerName"), " ").getItem(0)).withColumn("LastName", split(col("CustomerName"), " ").getItem(1))

# Filter and reorder columns
transformed_df = transformed_df["SalesOrderNumber", "SalesOrderLineNumber", "OrderDate", "Year", "Month", "FirstName", "LastName", "Email", "Item", "Quantity", "UnitPrice", "Tax"]

# Display the first five orders
transformed_df.show(5)

+----------------+--------------------+----------+----+-----+---------+---------+--------------------+--------------------+--------+---------+--------+
|SalesOrderNumber|SalesOrderLineNumber| OrderDate|Year|Month|FirstName| LastName|               Email|                Item|Quantity|UnitPrice|     Tax|
+----------------+--------------------+----------+----+-----+---------+---------+--------------------+--------------------+--------+---------+--------+
|         SO49171|                   1|2021-01-01|2021|    1|   Mariah|   Foster|mariah21@adventur...|  Road-250 Black, 48|       1|2181.5625| 174.525|
|         SO49172|                   1|2021-01-01|2021|    1|    Brian|   Howard|brian23@adventure...|    Road-250 Red, 44|       1|  2443.35| 195.468|
|         SO49173|                   1|2021-01-01|2021|    1|    Linda|  Alvarez|linda19@adventure...|Mountain-200 Silv...|       1|2071.4197|165.7136|
|         SO49174|                   1|2021-01-01|2021|    1|     Gina|Hernandez|gina4@a

In [31]:
transformed_df.write.mode("overwrite").parquet('Files/transformed_data/orders')

print ("Transformed data saved!")

Transformed data saved!


In [32]:
orders_df = spark.read.format("parquet").load("Files/transformed_data/orders")
orders_df.show(5)

+----------------+--------------------+----------+----+-----+---------+---------+--------------------+--------------------+--------+---------+--------+
|SalesOrderNumber|SalesOrderLineNumber| OrderDate|Year|Month|FirstName| LastName|               Email|                Item|Quantity|UnitPrice|     Tax|
+----------------+--------------------+----------+----+-----+---------+---------+--------------------+--------------------+--------+---------+--------+
|         SO49171|                   1|2021-01-01|2021|    1|   Mariah|   Foster|mariah21@adventur...|  Road-250 Black, 48|       1|2181.5625| 174.525|
|         SO49172|                   1|2021-01-01|2021|    1|    Brian|   Howard|brian23@adventure...|    Road-250 Red, 44|       1|  2443.35| 195.468|
|         SO49173|                   1|2021-01-01|2021|    1|    Linda|  Alvarez|linda19@adventure...|Mountain-200 Silv...|       1|2071.4197|165.7136|
|         SO49174|                   1|2021-01-01|2021|    1|     Gina|Hernandez|gina4@a

In [33]:
orders_df.write.partitionBy("Year","Month").mode("overwrite").parquet("Files/partitioned_data")

print ("Transformed data saved!")

Transformed data saved!


In [55]:
orders_2021_df = spark.read.format("parquet").load("Files/partitioned_data/Year=2021/Month=*")

orders_2021_df.show(5)

ConnectionRefusedError: [Errno 61] Connection refused

## SQL